# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import gmaps 
import requests
import os

# Import API key
from config import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,t'ongch'on-up,38.9539,127.8917,11.19,77,1,3.49,KP,2024-04-11 18:22:23
1,tanout,14.9709,8.8879,30.42,9,99,4.56,NE,2024-04-11 18:22:24
2,iqaluit,63.7506,-68.5145,-2.15,68,100,2.57,CA,2024-04-11 18:17:50
3,blackmans bay,-43.0167,147.3167,11.40,83,82,0.45,AU,2024-04-11 18:22:10
4,puerto ayora,-0.7393,-90.3518,28.98,97,48,4.47,EC,2024-04-11 18:22:24


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_map_plot = city_data_df.hvplot.points("Lng", 
                                      "Lat", 
                                      geo = True,
                                      color = "City",
                                      tiles = "OSM",
                                      size = "Humidity",
                                      scale = 0.6)

# Display the map
city_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df.loc[(city_data_df["Max Temp"] < 27)
                                  & (city_data_df["Max Temp"] > 21)
                                  & (city_data_df["Cloudiness"] ==0)
                                  & (city_data_df["Wind Speed"] < 4.5)]
                                  
# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
131,nova vicosa,-17.8919,-39.3719,26.35,81,0,3.19,BR,2024-04-11 18:22:51
193,lampasas,31.2168,-98.2336,25.72,33,0,4.02,US,2024-04-11 18:21:39
225,quebo,11.3333,-14.9333,24.42,63,0,3.45,GW,2024-04-11 18:23:06
305,namibe,-15.1961,12.1522,22.68,72,0,2.55,AO,2024-04-11 18:21:17
375,pyin oo lwin,22.0333,96.4667,21.62,32,0,0.35,MM,2024-04-11 18:23:34
523,timimoun,29.2639,0.2310,21.21,13,0,4.12,DZ,2024-04-11 18:24:01
529,pikine,14.7500,-17.4000,22.47,88,0,3.09,SN,2024-04-11 18:24:02
535,saint-pierre,-21.3393,55.4781,25.82,73,0,2.57,RE,2024-04-11 18:22:39
565,north ward,-19.2454,146.8075,21.95,78,0,0.00,AU,2024-04-11 18:24:10


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
def new_func():
    hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng"]].copy()
    return hotel_df

hotel_df = new_func()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Hotel Name
131,nova vicosa,BR,-17.8919,-39.3719,
193,lampasas,US,31.2168,-98.2336,
225,quebo,GW,11.3333,-14.9333,
305,namibe,AO,-15.1961,12.1522,
375,pyin oo lwin,MM,22.0333,96.4667,
523,timimoun,DZ,29.2639,0.2310,
529,pikine,SN,14.7500,-17.4000,
535,saint-pierre,RE,-21.3393,55.4781,
565,north ward,AU,-19.2454,146.8075,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
# Increased radius from 10000 metres to 30000 metres to obtain more hotels
radius = 10000
params = {"categories": "accommodation.hotel",
          "apiKey":geoapify_key,
          "limit":20}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
def new_func(index):
    hotel_df.loc[index, "Hotel Name"]

for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lon = row["Lng"]
    lat = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
nova vicosa - nearest hotel: Pousada Pontal da Barra
lampasas - nearest hotel: No hotel found
quebo - nearest hotel: No hotel found
namibe - nearest hotel: Nelsal Pensao
pyin oo lwin - nearest hotel: Hotel Pan Thitsar
timimoun - nearest hotel: أناف
pikine - nearest hotel: Residence Marodi TV
saint-pierre - nearest hotel: Tropic Hotel
north ward - nearest hotel: Aquarius on the Beach


,City,Country,Lat,Lng,Hotel Name
131,nova vicosa,BR,-17.8919,-39.3719,Pousada Pontal da Barra
193,lampasas,US,31.2168,-98.2336,No hotel found
225,quebo,GW,11.3333,-14.9333,No hotel found
305,namibe,AO,-15.1961,12.1522,Nelsal Pensao
375,pyin oo lwin,MM,22.0333,96.4667,Hotel Pan Thitsar
523,timimoun,DZ,29.2639,0.2310,أناف
529,pikine,SN,14.7500,-17.4000,Residence Marodi TV
535,saint-pierre,RE,-21.3393,55.4781,Tropic Hotel
565,north ward,AU,-19.2454,146.8075,Aquarius on the Beach


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:

# Configure the map plot

hotel_map_plot = hotel_df.hvplot.points(
            "Lng",
            "Lat",
            geo = True, 
            tiles = "OSM", 
            color = "City",
            frame_width = 600, 
            frame_height = 300,
            hover_cols = ["Hotel Name", "Country"],
            scale = 0.6)

# Display the map
hotel_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)